# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 88 (delta 37), reused 53 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (88/88), 33.58 KiB | 4.80 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_L24O_cv
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

2025-04-29 14:43:05.080110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745937785.342187      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745937785.417046      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from gmrrnet_adhd.models.ShallowConvNet  import ShallowConvNet 

model_name = 'ShallowConvNet'
model_args = {
    'Chans' : 19,
    'Samples' : 512,
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'version': '2018'
}

compile_args = {
    'loss': CategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'optimizer': 'adam',
    'metrics' : ['categorical_accuracy']
}

model = ShallowConvNet(**model_args)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745937803.358185      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745937803.358814      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast (Cast)                          │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_1 (Conv2D)                    │ (None, 19, 500, 40)         │             560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_2 (Conv2D)                    │ (None, 1, 500, 40)          │          30,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1, 500, 40)          │             160 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 1, 500, 40)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 67, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 67, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 67, 40)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2680)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │           5,362 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_activation (Activation)          │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,522 (142.66 KB)

 Trainable params: 36,442 (142.35 KB)

 Non-trainable params: 80 (320.00 B)

# Resultados - Leave 24 Subjects Out +  CV (cada fold se corre 5 veces)

In [5]:
import os

import pickle

with open("/kaggle/input/ieee-tdah-control-database/folds.pkl", "rb") as f:
    folds = pickle.load(f)

In [6]:
results = train_L24O_cv(ShallowConvNet, X, y, sbjs, model_args, compile_args, folds)

Fold 1/5. Test subjects: ['v28p', 'v274', 'v1p', 'v231', 'v22p', 'v29p', 'v206', 'v238', 'v31p', 'v35p', 'v177', 'v200', 'v112', 'v113', 'v48p', 'v140', 'v131', 'v125', 'v55p', 'v143', 'v43p', 'v305', 'v134', 'v114']


I0000 00:00:1745937808.673749      70 service.cc:148] XLA service 0x7cff240047a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745937808.674435      70 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745937808.674455      70 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745937809.067260      70 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745937813.199664      70 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Fold metrics: {'accuracy': 0.9032257795333862, 'recall': 0.8987068965517242, 'precision': 0.9218403547671841, 'kappa': 0.8043731020514271, 'auc': 0.8987068965517242}
Fold 2/5. Test subjects: ['v18p', 'v39p', 'v234', 'v32p', 'v190', 'v6p', 'v254', 'v204', 'v24p', 'v183', 'v246', 'v219', 'v298', 'v41p', 'v47p', 'v308', 'v52p', 'v300', 'v59p', 'v299', 'v302', 'v51p', 'v109', 'v127']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold metrics: {'accuracy': 0.6708633303642273, 'recall': 0.653980123792715, 'precision': 0.669977090345125, 'kappa': 0.31577232759290585, 'auc': 0.653980123792715}
Fold 3/5. Test subjects: ['v215', 'v3p', 'v209', 'v37p', 'v213', 'v15p', 'v284', 'v181', 'v19p', 'v34p', 'v263', 'v244', 'v138', 'v121', 'v46p', 'v54p', 'v120', 'v310', 'v147', 'v50p', 'v56p', 'v107', 'v297', 'v108']


...

Fold metrics: {'accuracy': 0.8947963714599609, 'recall': 0.869040518270918, 'precision': 0.9228769722648229, 'kappa': 0.7708545570849819, 'auc': 0.869040518270918}
Fold 4/5. Test subjects: ['v227', 'v8p', 'v236', 'v14p', 'v196', 'v27p', 'v33p', 'v179', 'v173', 'v10p', 'v265', 'v20p', 'v57p', 'v45p', 'v111', 'v115', 'v53p', 'v118', 'v123', 'v44p', 'v149', 'v303', 'v116', 'v151']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold metrics: {'accuracy': 0.9430942535400391, 'recall': 0.9403861379322215, 'precision': 0.9433276156318493, 'kappa': 0.8834979951399949, 'auc': 0.9403861379322215}
Fold 5/5. Test subjects: ['v279', 'v30p', 'v288', 'v286', 'v250', 'v12p', 'v38p', 'v25p', 'v21p', 'v40p', 'v198', 'v270', 'v117', 'v306', 'v309', 'v110', 'v42p', 'v58p', 'v307', 'v133', 'v304', 'v129', 'v49p', 'v60p']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold metrics: {'accuracy': 0.9828536510467529, 'recall': 0.9830676513411687, 'precision': 0.9827734407814994, 'kappa': 0.9656989907143319, 'auc': 0.9830676513411687}


In [7]:
results

[{'accuracy': 0.9032257795333862,
  'recall': 0.8987068965517242,
  'precision': 0.9218403547671841,
  'kappa': 0.8043731020514271,
  'auc': 0.8987068965517242},
 {'accuracy': 0.6708633303642273,
  'recall': 0.653980123792715,
  'precision': 0.669977090345125,
  'kappa': 0.31577232759290585,
  'auc': 0.653980123792715},
 {'accuracy': 0.8947963714599609,
  'recall': 0.869040518270918,
  'precision': 0.9228769722648229,
  'kappa': 0.7708545570849819,
  'auc': 0.869040518270918},
 {'accuracy': 0.9430942535400391,
  'recall': 0.9403861379322215,
  'precision': 0.9433276156318493,
  'kappa': 0.8834979951399949,
  'auc': 0.9403861379322215},
 {'accuracy': 0.9828536510467529,
  'recall': 0.9830676513411687,
  'precision': 0.9827734407814994,
  'kappa': 0.9656989907143319,
  'auc': 0.9830676513411687}]

In [8]:
import numpy as np

accs = []
for result in results:
    accs.append(result['accuracy'])

np.mean(accs)

0.8789666771888733

In [9]:
import pickle

with open(f'results_L24SO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)